# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [1]:
import os
import sys
from dotenv import load_dotenv, find_dotenv  # type: ignore

# ## Using the OpenAI LLM with the VectorStoreIndex
from openai import __version__ as openai_version  # type: ignore
from llama_index.core import __version__ as llama_index_version  # type: ignore

# Load environment variables
_ = load_dotenv(find_dotenv())  # read local .env file

print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai_version}")
print(f"llamaindex version: {llama_index_version}")

Python version: 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
OpenAI version: 1.17.1
llamaindex version: 0.10.29


In [2]:
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [3]:
with open("../docs/getting_started/starter_example.md", "r") as f:
    text = f.read()

Next, we create our LLM!

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [5]:
from llama_index.core import PromptTemplate

template = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)
text_qa_template = PromptTemplate(template)

refined_template = (
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)
qa_refined_template = PromptTemplate(refined_template)

Now, lets test a few questions!

## Text QA Template Testing

In [6]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To install LlamaIndex, you can follow the steps outlined in the context information provided:

1. Clone the LlamaIndex repository by running the following command in your terminal:
```bash
$ git clone https://github.com/jerryjliu/llama_index.git
```

2. Navigate to the cloned repository and verify its contents:
```bash
$ cd llama_index
$ ls
```

After completing these steps, you will have successfully installed LlamaIndex.


In [7]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index, you can follow these steps:

1. Download the LlamaIndex repository and navigate to the `examples/paul_graham_essay` folder.
2. Create a new `.py` file and import `VectorStoreIndex` and `SimpleDirectoryReader` from `llama_index`.
3. Use `SimpleDirectoryReader` to load the documents from the `data` folder.
4. Build the index using `VectorStoreIndex.from_documents(documents)`.
5. Use the index to create a query engine with `index.as_query_engine()`.
6. Query the index using `query_engine.query("Your query here")`.

By following these steps, you can create and query an index using LlamaIndex.


In [17]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [9]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = qa_refined_template.format(
    context_msg=text, query_str=question, existing_answer=existing_answer
)
response = llm.complete(prompt)
print(response.text)

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()
```


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [12]:
from llama_index.core.llms import ChatMessage

chat_history = [
    ChatMessage(
        role="system",
        content="You are a helpful QA chatbot that can answer questions about llama-index.",
    ),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index, you can follow these general steps:

1. Define the purpose of your index: Determine what you want your index to represent and what information you want it to include.

2. Choose the items to include in the index: Decide which elements or data points you want to track and include in your index.

3. Assign weights or values to the items: Determine the importance or relevance of each item in your index by assigning weights or values to them.

4. Calculate the index value: Once you have defined the items and their weights, calculate the index value based on the formula or methodology you have chosen.

5. Monitor and update the index: Regularly review and update your index to ensure it remains relevant and accurate over time.

When creating an index using llama-index, you can refer to the specific documentation and guidelines provided by llama-index for detailed instructions on how to create and manage your index.


## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.